# Importation des librairies

In [2]:
#Importation des librairies 
import pandas as pd 
import os
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.base import BaseEstimator, TransformerMixin
import category_encoders as ce
import joblib

In [3]:
os.chdir("/Users/pierrebourbon/Documents/PRO/Data/Master_SISE/Programmation_Python/Projet")

# Chargement des données 

In [4]:
df = pd.read_excel("extract-dpe.xlsx")

In [112]:
#Choix des variables explicatives

var_explicatives = ["Etiquette_DPE", 
                    "Type_bâtiment", 
                    "Année_construction", 
                    "Classe_inertie_bâtiment", 
                    "Hauteur_sous-plafond", 
                    "Surface_habitable_logement",  
                    "Type_énergie_principale_chauffage", 
                    "Isolation_toiture_(0/1)", "Code_INSEE_(BAN)"]

target = ["Conso_5_usages_é_finale"]

In [113]:
#Création d'un data frame avec les variables explicatives et la target
df_dpe = df[var_explicatives+target]

# Suppression des outliers, NA de la target et Type_batiment == immeuble

In [114]:
#Supression des outliers avec la méthode des quartiles [q1 - 1.5 * IQR , q3 + 1.5 * IQR]

#Sélection des vars quanti
df_quanti = df_dpe.select_dtypes(include=['number'])

#Calcul du IQR pour chaque colonne 
Quanti_bas = df_quanti.quantile(0.025)
Quanti_haut = df_quanti.quantile(0.975)
IQR = Quanti_haut-Quanti_bas

#Filtrage des données sans outliers 
df_dpe_filtered = df_dpe[~((df_quanti < (Quanti_bas - 1.5 * IQR)) | (df_quanti > (Quanti_haut + 1.5 * IQR))).any(axis=1)]

df_dpe_filtered.describe()

,Année_construction,Hauteur_sous-plafond,Surface_habitable_logement,Isolation_toiture_(0/1),Conso_5_usages_é_finale
count,186204.000000,313695.000000,311641.000000,186839.000000,313687.000000
mean,1975.358322,2.589259,65.811322,0.355156,10499.919965
std,28.364826,0.235743,34.695060,0.478562,9562.245845
min,1731.000000,1.100000,1.000000,0.000000,306.300000
25%,1958.000000,2.500000,44.000000,0.000000,4999.500000
50%,1973.000000,2.500000,63.000000,0.000000,7982.600000
75%,1998.000000,2.500000,79.700000,1.000000,12970.100000
max,2024.000000,4.600000,365.200000,1.000000,111399.400000


In [115]:
#On enlève les NA de la conso 
df_dpe_filtered = df_dpe_filtered.dropna(subset=["Conso_5_usages_é_finale"])
df_dpe_filtered.isnull().sum()

Etiquette_DPE                             0
Type_bâtiment                             0
Année_construction                   127491
Classe_inertie_bâtiment                2063
Hauteur_sous-plafond                      0
Surface_habitable_logement             2054
Type_énergie_principale_chauffage     10547
Isolation_toiture_(0/1)              126856
Code_INSEE_(BAN)                          0
Conso_5_usages_é_finale                   0
dtype: int64

In [116]:
#On enlève les "immeubles"
df_dpe_filtered = df_dpe_filtered[df_dpe_filtered['Type_bâtiment'] != 'immeuble']
df_dpe_filtered.shape

(311814, 10)

# Train / Test Split

In [117]:
#Séparation variables explicatives et variable cible 
X = df_dpe_filtered[var_explicatives]
y = df_dpe_filtered[target]

X.head()

,Etiquette_DPE,Type_bâtiment,Année_construction,Classe_inertie_bâtiment,Hauteur_sous-plafond,Surface_habitable_logement,Type_énergie_principale_chauffage,Isolation_toiture_(0/1),Code_INSEE_(BAN)
0,D,appartement,1969.0,Légère,2.5,90.5,Réseau de Chauffage urbain,NaN,69256
1,E,appartement,1960.0,Moyenne,2.5,56.8,Réseau de Chauffage urbain,NaN,69256
2,D,appartement,1975.0,Lourde,2.5,83.6,Réseau de Chauffage urbain,NaN,69256
3,F,appartement,1948.0,Lourde,2.5,55.7,Gaz naturel,NaN,69256
4,C,appartement,2013.0,Moyenne,2.5,70.2,Gaz naturel,NaN,69256


In [118]:
#Train / Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

X_train.shape

(218269, 9)

# Pipeline 

In [119]:
#Colonnes numériques et catégorielles / traitement spécifique pour isolation toiture 
num_features = ["Année_construction", "Hauteur_sous-plafond", "Surface_habitable_logement"]
iso_feature = ["Isolation_toiture_(0/1)"]
cat_features = ["Etiquette_DPE", "Type_bâtiment", "Classe_inertie_bâtiment", "Type_énergie_principale_chauffage", "Code_INSEE_(BAN)"]

In [120]:
X[num_features]

,Année_construction,Hauteur_sous-plafond,Surface_habitable_logement
0,1969.0,2.5,90.5
1,1960.0,2.5,56.8
2,1975.0,2.5,83.6
3,1948.0,2.5,55.7
4,2013.0,2.5,70.2
...,...,...,...
318489,2013.0,2.5,42.0
318490,NaN,2.3,34.6
318491,NaN,2.5,50.0
318492,NaN,2.5,59.2


In [122]:
# Transformateur personnalisé pour convertir la colonne isolation toiture en type `str`
class ConvertToStrTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # Rien à ajuster
    
    def transform(self, X):
        # Conversion de la colonne en type `str`
        return X.astype(str)

In [123]:
#Transformateur pour le target encoder 

class TargetEncodingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, cols=None):
        self.cols = cols
        self.encoder = ce.TargetEncoder(cols=self.cols)
    
    def fit(self, X, y):
        self.encoder.fit(X, y)
        return self
    
    def transform(self, X):
        return self.encoder.transform(X)

In [124]:
#Pipeline pour les colonnes numériques 
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])

In [125]:
#Pipeline pour la colonne isolation toiture 
iso_transformer = Pipeline(steps=[
    ('to_object', ConvertToStrTransformer()),
    ('imputer', SimpleImputer(strategy='constant', fill_value='Inconnue'))
])

In [126]:
#Pipeline pour les colonnes catégorielles
cat_transformer = Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder', TargetEncodingTransformer())
])

In [127]:
#Prepocessor 
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('iso', iso_transformer, iso_feature),
        ('cat', cat_transformer, cat_features + iso_feature)
    ]
)

In [128]:
#Pipeline final avec la preprocessor et le modèle 
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regression', DecisionTreeRegressor(max_depth=10, min_samples_leaf=4, min_samples_split=10))
])

In [129]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [130]:
# Pipeline final avec les données encodées et imputées
pipeline.fit(X_train, y_train)

# Sauvegarde du pipeline complet
joblib.dump(pipeline, 'pipeline_ml_regression.joblib')

['pipeline_ml_regression.joblib']